In [11]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('io')

# Waylay Python SDK - Making REST calls

The Waylay Python SDK exposes a number of _REST Services_, each a collection of _REST Resources_ that have _action methods_.

For an overview, see 
> `[IO]` https://docs-io.waylay.io/#/api/sdk/python<br>
> `[Enterprise]` https://docs.waylay.io/api/sdk/python/

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform.

In general, the Python SDK takes care of
* authentication
* the endpoint url to call, and HTTP method to use
* converting JSON responses to Python objects, extracting the relevant data
* converting Python request body objects to JSON
* handling errors 

Check out the documentation sites ([io](https://docs-io.waylay.io/#/api/sdk/python), [enterprise](https://docs.waylay.io/api/sdk/python/)) on

* how requests and response are exactly represented as json
* which additional parameters can be provided
* what the preconditions and effects are of your call

### request arguments
These are the general rules for making requests, corresponding to the REST documentation of the underlying REST action:
* url **path parameters** are passed as **positional arguments**:
  ###### example

  > `waylay_client.analytics.query.get('151CF-temperature')` 

  will use 151CF-temperature to construct a request to fetch this query definition

  > `GET .../config/query/151CF-temperature` 

* **query parameters** belong in a named **`params` argument**:
  ###### example
  > `waylay_client.analytics.query.data('151CF-temperature', params={'from':'2021-03-01T10:00:00+00:00'})`
  
  will bind the `from` parameter

* **request objects** are passed into a **`body` argument**:
  ###### example
  > `waylay_client.analytics.query.execute(body={'resource':'RDJ_89839','metric':'revolutions'})`}|

### response handling
These are the general rules on how the SDK treats results of REST calls:
* Only successfull responses (HTTP status `2XX`) return a result, other responses raise an exception.
* The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour.
* When the REST call produces **timeseries data**, the SDK will return a **pandas DataFrame**.
* By providing the  `raw=True` argument, you instruct the SDK to skip all error and response handling, and **just return the HTTP response**.

### error handling

The SDK uses the following exception hierarchy to notify problems. These exception classes belong to the `waylay.exception` module. They all descend from a `WaylayError` base class.

| exception class | raised in case of |
| --------------- | ----------- |
| `AuthError` | Waylay authentication errors. |
| `ConfigError` |  Waylay client configuration errors. |
| `RequestError` | Errors in tools and utilities that are not directly related to a REST call. |
| `RestRequestError` | Failure to prepare a REST call. |
| `RestResponseError` | Wraps the result of a failed REST call. |
| `RestResponseParseError` | Failure to parse the result of a succesfull REST call. |

Errors of type `RestResponse` have the following attributes that you can use to handle problems:

* `response` contains the full HTTP Response object of the REST call, which lets you inspect the status code (`response.status_code`) , response body (`response.body`) and other attributes such as _headers_.
* `message` will give you the most relevant error message




### http response information

Most _action methods_ support a `raw=True` parameter. This will prevent exeption handling and parsing of the REST call by the SDK. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [12]:
http_response = waylay_client.analytics.query.get('151CF-temperature', raw=True)
{
    'url':http_response.url,
    'method':http_response.method,
    'content-type':http_response.headers['content-type'],
    'status_code':http_response.status_code,
    'name':http_response.body['name'],
}


AnalyticsActionError: AnalyticsActionError(404: 'operation=not_found_error'; GET 'https://ts-analytics-io.waylay.io/config/query/151CF-temperature?api_version=0.19')

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [3]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.analytics.query.get('where are you???')
except RestResponseError as exc:
   print(exc.message)
   print(exc.response.status_code)
   print(exc.response.url)
    


operation=not_found_error
404
https://ts-analytics.waylay.io/config/query/where%20are%20you?api_version=0.19


## accessing the complete response
The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour

In [4]:
# the `query` entity in the REST response is given as default response on the SDK action
waylay_client.analytics.query.get('151CF-temperature')

{'data': [{'metric': 'temperature',
   'resource': '151CF',
   'aggregation': 'median'},
  {'metric': 'temperature', 'resource': '151D8', 'aggregation': 'median'}],
 'freq': 'P1D',
 'window': 'P14D'}

In [5]:
waylay_client.analytics.query.get('151CF-temperature', select_path=False)

{'messages': [],
 'name': '151CF-temperature',
 'query': {'data': [{'metric': 'temperature',
    'resource': '151CF',
    'aggregation': 'median'},
   {'metric': 'temperature', 'resource': '151D8', 'aggregation': 'median'}],
  'freq': 'P1D',
  'window': 'P14D'},
 'meta': None,
 'attrs': {'created': '2020-08-29T09:01:34.566657+00:00',
  'created_by': 'users/dcf8612b-94fa-4cd4-85fb-e66a1724712a',
  'modified': '2021-03-05T14:15:40.405655+00:00',
  'modified_by': 'users/dcf8612b-94fa-4cd4-85fb-e66a1724712a'},
 '_links': {'self': {'href': 'https://ts-analytics.waylay.io/config/query/151CF-temperature'}}}

In [6]:
waylay_client.analytics.query.get('151CF-temperature', select_path=['attrs','modified'])

'2021-03-05T14:15:40.405655+00:00'

## customise or replace dataframe conversions
The methods of the `analytics.query` that return timeseries data, will normally return their data as a pandas `DataFrame`. These have immediate display support in jupyter notebooks:

In [7]:
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P5D', 'until':'2020-01-31'}
)

resource,151CF,151D8
metric,temperature,temperature
aggregation,median,median
timestamp,,
2020-01-26 00:00:00+00:00,17,16
2020-01-27 00:00:00+00:00,21,21
2020-01-28 00:00:00+00:00,21,20
2020-01-29 00:00:00+00:00,21,20
2020-01-30 00:00:00+00:00,21,20


You can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optional parameter.

A falsy value will let the method return just the json data payload (as python object).

In [8]:
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P5D', 'until':'2020-01-31'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': '151CF', 'metric': 'temperature', 'aggregation': 'median'},
   {'resource': '151D8', 'metric': 'temperature', 'aggregation': 'median'}],
  'data': [[1579996800000, 17, 16],
   [1580083200000, 21, 21],
   [1580169600000, 21, 20],
   [1580256000000, 21, 20],
   [1580342400000, 21, 20]],
  'data_axis': 'column',
  'attributes': {'role': 'input'},
  'window_spec': {'from': 1579996800000,
   'until': 1580428800000,
   'window': 'P5D',
   'freq': 'P1D'}}]

In [9]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P20D', 'until':'2020-01-31'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1578700800000: [17.0, 16.0],
 1578787200000: [17.0, 16.0],
 1578873600000: [21.0, 20.0],
 1578960000000: [21.0, 20.0],
 1579046400000: [21.0, 20.0],
 1579132800000: [21.0, 20.5],
 1579219200000: [21.0, 20.0],
 1579305600000: [17.0, 16.0],
 1579392000000: [17.0, 16.0],
 1579478400000: [21.0, 20.0],
 1579564800000: [20.0, 20.0],
 1579651200000: [20.5, 20.0],
 1579737600000: [21.0, 20.0],
 1579824000000: [20.5, 20.0],
 1579910400000: [17.0, 16.0],
 1579996800000: [17.0, 16.0],
 1580083200000: [21.0, 21.0],
 1580169600000: [21.0, 20.0],
 1580256000000: [21.0, 20.0],
 1580342400000: [21.0, 20.0]}

In [10]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P20D', 'until':'2020-01-31'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1.5787008e+12, 1.5787872e+12, 1.5788736e+12, 1.5789600e+12,
        1.5790464e+12, 1.5791328e+12, 1.5792192e+12, 1.5793056e+12,
        1.5793920e+12, 1.5794784e+12, 1.5795648e+12, 1.5796512e+12,
        1.5797376e+12, 1.5798240e+12, 1.5799104e+12, 1.5799968e+12,
        1.5800832e+12, 1.5801696e+12, 1.5802560e+12, 1.5803424e+12],
       [1.7000000e+01, 1.7000000e+01, 2.1000000e+01, 2.1000000e+01,
        2.1000000e+01, 2.1000000e+01, 2.1000000e+01, 1.7000000e+01,
        1.7000000e+01, 2.1000000e+01, 2.0000000e+01, 2.0500000e+01,
        2.1000000e+01, 2.0500000e+01, 1.7000000e+01, 1.7000000e+01,
        2.1000000e+01, 2.1000000e+01, 2.1000000e+01, 2.1000000e+01],
       [1.6000000e+01, 1.6000000e+01, 2.0000000e+01, 2.0000000e+01,
        2.0000000e+01, 2.0500000e+01, 2.0000000e+01, 1.6000000e+01,
        1.6000000e+01, 2.0000000e+01, 2.0000000e+01, 2.0000000e+01,
        2.0000000e+01, 2.0000000e+01, 1.6000000e+01, 1.6000000e+01,
        2.1000000e+01, 2.0000000e+01, 2.000000